<a href="https://colab.research.google.com/github/satoruk-icepp/MEG2XEC/blob/master/MEG2Reg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
%%writefile .comet.config
[comet]
api_key=mIel5ZAPOioTs0Cij75dSSQXs
logging_file = /tmp/comet.log
logging_file_level = info


Overwriting .comet.config


In [27]:
! [ ! -z "$COLAB_GPU" ] && pip install skorch comet_ml

In [0]:
from comet_ml import Experiment
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as utils
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR
from tqdm import tqdm, tqdm_notebook
from scipy.optimize import curve_fit

In [29]:
experiment = Experiment(project_name="CWreg")
curtime = datetime.now().strftime("%Y%m%d-%H%M%S")
experiment.set_name("%s"%(curtime))
params={'batch_size' : 200,
        'learning_rate':0.1,
        'dropout_conv':0.05,
        'dropout_fc':0.05,
        'optim':"Adam",
        'weight_decay':1e-05,
        'Nresblock':4,
        'Nsd':0,
        'Wthreshold':np.log(0.2+1e-02)/2.5+1,
        'weightstd':0.0001,
        'Nlayer':32,
        'Nfc':0,
        'Nfcnodes':256,
        'LRgamma':0.1
}
experiment.log_parameters(params)

COMET INFO: ----------------------------
COMET INFO: Comet.ml Experiment Summary:
COMET INFO:   Data:
COMET INFO:     url: https://www.comet.ml/satoruk-icepp/cwreg/fa8ea0d5e900426882c264b9cf12ca5a
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     sys.cpu.percent.01 [2]               : (13.5, 18.8)
COMET INFO:     sys.cpu.percent.02 [2]               : (7.9, 10.7)
COMET INFO:     sys.cpu.percent.03 [2]               : (7.9, 11.3)
COMET INFO:     sys.cpu.percent.04 [2]               : (11.8, 83.2)
COMET INFO:     sys.cpu.percent.avg [2]              : (11.600000000000001, 29.675)
COMET INFO:     sys.gpu.0.free_memory [2]            : (17071734784.0, 17071734784.0)
COMET INFO:     sys.gpu.0.gpu_utilization [2]        : (0.0, 0.0)
COMET INFO:     sys.gpu.0.total_memory               : (17071734784.0, 17071734784.0)
COMET INFO:     sys.gpu.0.used_memory [2]            : (0.0, 0.0)
COMET INFO:     sys.ram.total [2]                    : (27395411968.0, 27395411968.0)
COMET INFO:     s

In [0]:
device = torch.device('cuda:0')

In [0]:
# csv_data  = pd.read_csv('/content/drive/My Drive/MEG2CW/fout.csv')
# names=['E','U','V','W','PT','PP','UR','VR','WR']
# PMnames = ['PM%d'%(i) for i in range(4760)]
# names = names+PMnames
# csv_data  = pd.read_csv('/content/drive/My Drive/MEG2CW/fout_norm.csv',names=names)
csv_data  = pd.read_csv('/content/drive/My Drive/MEG2CW/fout_norm.csv')

In [0]:
print(csv_data.shape)
print(params['Wthreshold'])
# csv_data = csv_data[csv_data['w']<params['Wthreshold']]
csv_data

In [0]:
csv_data_numpy = csv_data.to_numpy()
del csv_data

In [0]:
Energy  = csv_data_numpy[:,0]
UVW     = csv_data_numpy[:,1:4]
DIR     = csv_data_numpy[:,4:6]
UVWREC  = csv_data_numpy[:,6:9]
PMResponse = csv_data_numpy[:,9:4101]

In [0]:
Energy     = Energy.reshape(-1,1)
PMResponse = PMResponse.reshape(-1,93,44)
indx_U = np.arange(-21.5,22.5)
indx_V = np.arange(-46,47)
# for i in np.arange(-21.5,22.5):
PMU = (np.dot(np.sum(PMResponse[:],axis=1),indx_U)/np.sum(np.sum(PMResponse[:],axis=1),axis=1))/22
PMV = (np.dot(np.sum(PMResponse[:],axis=2),indx_V)/np.sum(np.sum(PMResponse[:],axis=2),axis=1))/46.5
PMU = PMU.reshape(-1,1)
PMV = PMV.reshape(-1,1)
PMURMS = np.sqrt(np.mean(np.square(np.sum(PMResponse[:],axis=1)),axis=1)).reshape(-1,1)
PMVRMS = np.sqrt(np.mean(np.square(np.sum(PMResponse[:],axis=2)),axis=1)).reshape(-1,1)
plt.hist2d(UVW[:,2],PMURMS.reshape(-1),bins=[40,40])
ADD = np.concatenate((PMU,PMV,PMURMS,PMVRMS),axis=1)
# print(ADD)

In [0]:
del csv_data_numpy

In [0]:
PMResponse = PMResponse.reshape(-1,4092)
# PMResponse = PMResponse/PMResponseScale

In [0]:
Energy        = torch.tensor(Energy).float()
UVW           = torch.tensor(UVW).float()
DIR           = torch.tensor(DIR).float()
ADD           = torch.tensor(ADD).float()
UVWREC        = torch.tensor(UVWREC).float()
PMResponse    = torch.tensor(PMResponse).float()

In [0]:
from torch.utils.data.dataset import Subset
BATCH_SIZE = params["batch_size"]
calo_dataset    = utils.TensorDataset(Energy,UVW,UVWREC,DIR,ADD,PMResponse)
data_size =  len(calo_dataset)
full_size = int(data_size/1000)*1000
print(data_size)
train_dataset = Subset(calo_dataset,list(range(0,int(0.8*full_size))))
val_dataset = Subset(calo_dataset,list(range(int(0.8*full_size),full_size)))
train_dataloader = torch.utils.data.DataLoader(train_dataset, 
                                              batch_size=BATCH_SIZE, 
                                              pin_memory=True, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, 
                                              batch_size=len(val_dataset), 
                                              pin_memory=True, shuffle=True)

In [0]:
def normal_init(m, mean, std):
    if isinstance(m, (nn.Linear, nn.Conv2d, nn.BatchNorm2d, nn.BatchNorm1d)):
        m.weight.data.normal_(mean, std)
        if m.bias.data is not None:
            m.bias.data.zero_()

In [0]:
class ResidualBlock(nn.Module):
    def __init__(self,input_size):
        super(ResidualBlock, self).__init__()        
        self.conv1 = nn.Conv2d(input_size,input_size,3,padding=1)
        self.conv2 = nn.Conv2d(input_size,input_size,3,padding=1)
        self.bn1 = nn.BatchNorm2d(self.conv1.out_channels)
        self.bn2 = nn.BatchNorm2d(self.conv2.out_channels)        
        self.activation = nn.LeakyReLU(0.0)
    def forward(self,xraw):
        x = self.activation(self.bn1(self.conv1(xraw)))
        x = self.activation(self.bn2(self.conv2(x))+xraw)
        return x

In [0]:
class Regressor(nn.Module):
    def __init__(self, dropout_conv =0.0,dropout_fc=0.0,Nresblock=0,Nsd=0,Nlayer=32,Nfc=4,Nfcnodes=256):
        super(Regressor, self).__init__()
        # self.fc1 = nn.Linear(4092,256)
        # self.conv1 = nn.Conv2d(1, Nlayer, kernel_size=(10,10), stride = 5, padding = (1,3))#(93+6,44+6)->24,12
        # self.conv2 = nn.Conv2d(
        #     self.conv1.out_channels, 
        #     self.conv1.out_channels*2, 
        #     kernel_size=(4, 3), 
        #     stride=2
        #     # ,padding = (1,1)
        # )#24*12->12*6
        # self.conv3 = nn.Conv2d(
        #     self.conv2.out_channels, 
        #     self.conv2.out_channels*2, 
        #     3
        #     # ,stride=2
        # )#12*6->6*3
        self.conv1 = nn.Conv2d(1, Nlayer, kernel_size=(7,6), stride = 4, padding = (3,3))#(93+6,44+6)->24,12
        self.conv2 = nn.Conv2d(
            self.conv1.out_channels, 
            self.conv1.out_channels*2, 
            kernel_size=(4, 4), 
            stride=2
            ,padding = (1,1)
        )#24*12->12*6
        self.conv3 = nn.Conv2d(
            self.conv2.out_channels, 
            self.conv2.out_channels*2, 
            2
            ,stride=2
        )#12*6->6*3
        self.convsd = nn.Conv2d(
            self.conv1.out_channels, 
            self.conv1.out_channels, 
            3,
            padding=1
        )#12*6->6*3 
        self.rb = ResidualBlock(self.conv1.out_channels)
        self.fcstart = nn.Linear(self.conv3.out_channels*18+7,Nfcnodes)
        self.Nfc = Nfc
        self.fc=[nn.Linear(self.fcstart.out_features//2**i,self.fcstart.out_features//2**(i+1)).to(device) for i in range(self.Nfc)]
        self.fcend = nn.Linear(self.fcstart.out_features//2**(Nfc),3)
        
        self.bn1 = nn.BatchNorm2d(self.conv1.out_channels)
        self.bn2 = nn.BatchNorm2d(self.conv2.out_channels)
        self.bn3 = nn.BatchNorm2d(self.conv3.out_channels)
        self.dropout1 = nn.Dropout(dropout_conv)
        self.dropoutfc = nn.Dropout(dropout_fc)
        self.Nresblock = Nresblock
        self.Nsd = Nsd
        
    def forward(self, x, uvw_rec):
        # x = F.relu(self.dropout_fc1(self.fc1(x)))
        x = x.view(x.shape[0],1,-1)
        
        # x_mppc,x_pmt = x.split(4092)
        x = x.view(x.shape[0],1,93,44)
        x = F.relu(self.bn1(self.conv1(x)))
        for i in range(self.Nresblock):
            x = self.dropout1(self.rb(x))
        for i in range(self.Nsd):
            x = F.relu(self.dropout1(self.bn1(self.convsd(x))))
        x = F.relu(self.dropout1(self.bn2(self.conv2(x))))
        x = F.relu(self.dropout1(self.bn3(self.conv3(x))))
        
        x = x.view(x.shape[0],self.conv3.out_channels*18)
        x = torch.cat([x,uvw_rec],dim=1)
        x = F.relu(self.fcstart(x))
        for i in range(self.Nfc):
            x = F.relu(self.dropoutfc(self.fc[i](x)))
        # x = F.relu(self.dropoutfc(self.fc3(x)))
        # x = F.relu(self.dropoutfc(self.fc4(x)))
        x = self.fcend(x)
        return torch.tanh(x)
    def weight_init(self, mean, std):
        for m in self._modules:
            normal_init(self._modules[m], mean, std)

In [0]:
model = Regressor(
    params["dropout_conv"],
    params["dropout_fc"],
    params["Nresblock"],
    params["Nsd"],
    params['Nlayer'],
    params['Nfc'],
    params['Nfcnodes']
    ).to(device)
print(model)
# model.weight_init(mean=0.0, std=params['weightstd'])

In [0]:
# learning_rate = 0.001
# opt = optim.Adam(regressor.parameters(), lr=learning_rate)
opt = optim.Adam(model.parameters(), lr=params["learning_rate"], weight_decay=params["weight_decay"])
scheduler = MultiStepLR(opt, milestones=[50,250,650], gamma=params["LRgamma"])

In [0]:
Energy_mean, UVW_mean, DIR_mean = Energy.mean(dim=0).to(device), UVW.mean(dim=0).to(device), DIR.mean(dim=0).to(device)
EZRP_mean = torch.cat([Energy_mean, UVW_mean]).to(device)
UVWDIR_mean = torch.cat([UVW_mean, DIR_mean]).to(device)
def metric_relative_mse(y_pred,y_true):
    y_true_mean = y_true.mean(dim=0)
    # return (((y_true - y_pred).pow(2).mean(dim=0) / (y_true - EZRP_mean).pow(2).mean(dim=0)).sum()).sqrt()
    return (((y_true - y_pred).pow(2).mean(dim=0) / (y_true - UVW_mean).pow(2).mean(dim=0)).sum()).sqrt()
    # return (((y_true[:,2] - y_pred[:,2]).pow(2).mean(dim=0) / (y_true[:,2] - UVW_mean[2]).pow(2).mean(dim=0)).sum()).sqrt()
    # return (((y_true - y_pred).pow(2).mean(dim=0) / (y_true - UVWDIR_mean).pow(2).mean(dim=0)).sum()).sqrt()
    # return (((y_true - y_pred).pow(2).mean(dim=0) / (y_true - Energy_mean).pow(2).mean(dim=0)).sum()).sqrt()

In [0]:
# loss_fn = torch.nn.SmoothL1Loss().to(device)
loss_fn = torch.nn.L1Loss().to(device)

In [0]:
def run_training(epochs=100):
    # iterating over epochs...
    for epoch in tqdm(range(epochs)):
        first = True
        for Energy_b, UVW_b, UVWREC_b, DIR_b,ADD_b, PMResponse_b in train_dataloader:
        # moving them to device(for example, cuda-device)
            Energy_b, UVW_b, UVWREC_b, DIR_b,ADD_b, PMResponse_b = Energy_b.to(device), \
                                            UVW_b.to(device), \
                                            UVWREC_b.to(device), \
                                            DIR_b.to(device), \
                                            ADD_b.to(device), \
                                            PMResponse_b.to(device)

#             pred = regressor(EnergyDeposit_b)
            model.train()
            UVWDIR_b = torch.cat([UVW_b,DIR_b],dim=1)
            UVWDIRrec_b = torch.cat([UVWREC_b,DIR_b],dim=1)
            # pred = model(PMResponse_b,UVWREC_b)
            # pred = model(PMResponse_b,torch.zeros(UVW_b.shape[0],3).to(device))
            pred = model(PMResponse_b,torch.cat([ADD_b,UVWREC_b],dim=1))
            # loss        = loss_fn(pred, torch.cat([ParticleMomentum_b, ParticlePoint_b], dim=1))
            # loss        = loss_fn(pred, torch.cat([Energy_b,ZRP_b],dim=1))
            loss        = loss_fn(pred, UVW_b)
            loss_rec    = loss_fn(UVWREC_b, UVW_b)
            # loss        = loss_fn(pred[:,2], UVW_b[:,2])
            # loss_rec    = loss_fn(UVWREC_b[:,2], UVW_b[:,2])
            # loss        = loss_fn(pred, Energy_b)

            # model.train()
            opt.zero_grad()
            loss.backward()
            opt.step()
        
        with torch.no_grad():
            model.eval()
            pred = model(PMResponse_b,torch.cat([ADD_b,UVWREC_b],dim=1))
            train_mse       = metric_relative_mse(pred, UVW_b).item()
            train_mse_rec   = metric_relative_mse(UVWREC_b, UVW_b).item()

        if epoch%10==0:
            plt.figure(figsize=(20,12))
            grid = plt.GridSpec(3, 3, wspace=0.4, hspace=0.3)
            for idim in range(3):
                plt.subplot(grid[2,idim])
                plt.hist2d(UVW_b[:,2].cpu().detach().numpy(),UVW_b[:,idim].cpu().detach().numpy()-pred[:,idim].cpu().detach().numpy(),
                        range=[[-1,1],[-0.2,0.2]],
                        bins = [40,40]
                        )
        
        experiment.log_metric("learning_rate", scheduler.get_lr(),step=epoch)
        experiment.log_metric("train_loss", loss.item(),step=epoch)
        experiment.log_metric("train_mse", train_mse,step=epoch)
        experiment.log_metric("train_loss_rec", loss_rec.item(),step=epoch)
        experiment.log_metric("train_mse_rec", train_mse_rec,step=epoch)
        scheduler.step()
        for Energy_b, UVW_b, UVWREC_b, DIR_b,ADD_b, PMResponse_b in val_dataloader:
    # moving them to device(for example, cuda-device)
            Energy_b, UVW_b, UVWREC_b, DIR_b,ADD_b, PMResponse_b = Energy_b.to(device), \
                                            UVW_b.to(device), \
                                            UVWREC_b.to(device), \
                                            DIR_b.to(device), \
                                            ADD_b.to(device), \
                                            PMResponse_b.to(device)

            break

#             pred = regressor(EnergyDeposit_b)
        with torch.no_grad():
            model.eval()
            UVWDIR_b = torch.cat([UVW_b,DIR_b],dim=1)
            UVWDIRrec_b = torch.cat([UVWREC_b,DIR_b],dim=1)
            # pred = model(PMResponse_b,UVWREC_b)
            # pred = model(PMResponse_b,torch.zeros(UVW_b.shape[0],3).to(device))
            pred = model(PMResponse_b,torch.cat([ADD_b,UVWREC_b],dim=1))
            val_loss        = loss_fn(pred, UVW_b)
            val_loss_rec    = loss_fn(UVWREC_b, UVW_b)
            val_mse       = metric_relative_mse(pred, UVW_b).item()
            val_mse_rec   = metric_relative_mse(UVWREC_b, UVW_b).item()
            # val_mse   = metric_relative_mse(pred, Energy_b).item()
            # val_mse   = 0
        experiment.log_metric("val_loss", val_loss.item(),step=epoch)
        experiment.log_metric("val_loss_rec", val_loss_rec.item(),step=epoch)
        experiment.log_metric("val_mse", val_mse,step=epoch)
        experiment.log_metric("val_mse_rec", val_mse_rec,step=epoch)
        resolution     =[np.sqrt(np.mean((UVW_b[:,idim].cpu().detach().numpy()-pred[:,idim].cpu().detach().numpy())**2)) for idim in range(3)]
        resolution_lpf =[np.sqrt(np.mean((UVW_b[:,idim].cpu().detach().numpy()-UVWREC_b[:,idim].cpu().detach().numpy())**2)) for idim in range(3)]
        experiment.log_metric("u_resolution", resolution[0],step=epoch)
        experiment.log_metric("v_resolution", resolution[1],step=epoch)
        experiment.log_metric("w_resolution", resolution[2],step=epoch)
        experiment.log_metric("u_resolution_lpf", resolution_lpf[0],step=epoch)
        experiment.log_metric("v_resolution_lpf", resolution_lpf[1],step=epoch)
        experiment.log_metric("w_resolution_lpf", resolution_lpf[2],step=epoch)
        if epoch%10==0:
            # plt.figure(figsize=(20,12))
            # grid = plt.GridSpec(2, 3, wspace=0.4, hspace=0.3)
            for idim in range(3):
                plt.subplot(grid[0,idim])
                plt.hist(UVW_b[:,idim].cpu().detach().numpy()-pred[:,idim].cpu().detach().numpy(),range=(-0.1,0.1),bins=80)
                plt.title("%f"%(np.sqrt(np.mean((UVW_b[:,idim].cpu().detach().numpy()-pred[:,idim].cpu().detach().numpy())**2))))
                plt.subplot(grid[1,idim])
                plt.hist2d(UVW_b[:,2].cpu().detach().numpy(),UVW_b[:,idim].cpu().detach().numpy()-pred[:,idim].cpu().detach().numpy(),
                        range=[[-1,1],[-0.1,0.1]],
                        bins = [40,40]
                        )
            
            experiment.log_figure(figure=plt)
            plt.close()
        

In [0]:
with experiment.train():
    run_training(10000)